In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
7,Language Issues Could Prolong Wait For COVID-1...,https://www.wbez.org/stories/the-wait-for-a-co...,"Thu, 18 Feb 2021 12:00:00 GMT",2021-02-18 12:00:00+00:00
1,The Road to COVID-19 Immunity: Building Trust ...,https://reliefweb.int/report/world/road-covid-...,"Thu, 18 Feb 2021 11:36:00 GMT",2021-02-18 11:36:00+00:00
3,Some retirement communities lure new residents...,https://www.nbcnews.com/health/aging/some-reti...,"Thu, 18 Feb 2021 11:16:00 GMT",2021-02-18 11:16:00+00:00
5,Update on COVID-19 Outbreak at Inverness Jail ...,https://multco.us/multnomah-county/news/update...,"Thu, 18 Feb 2021 01:54:42 GMT",2021-02-18 01:54:42+00:00
6,FLASH REPORT #148 - COVID-19 Response and Reco...,https://www.sanjoseca.gov/Home/Components/News...,"Wed, 17 Feb 2021 22:11:40 GMT",2021-02-17 22:11:40+00:00
0,Safety and Efficacy of the BNT162b2 mRNA Covid...,https://www.nejm.org/doi/full/10.1056/NEJMc203...,"Wed, 17 Feb 2021 22:00:51 GMT",2021-02-17 22:00:51+00:00
2,Delayed Second Dose versus Standard Regimen fo...,https://www.nejm.org/doi/full/10.1056/NEJMclde...,"Wed, 17 Feb 2021 22:00:51 GMT",2021-02-17 22:00:51+00:00
4,Emergency Intubation in Covid-19 - nejm.org n...,https://www.nejm.org/doi/full/10.1056/NEJMvcm2...,"Wed, 17 Feb 2021 22:00:51 GMT",2021-02-17 22:00:51+00:00
8,Press Briefing by White House COVID-19 Respons...,https://www.whitehouse.gov/briefing-room/press...,"Wed, 17 Feb 2021 19:23:05 GMT",2021-02-17 19:23:05+00:00
9,"Coronavirus (COVID-19) Update: February 16, 20...",https://www.fda.gov/news-events/press-announce...,"Tue, 16 Feb 2021 21:27:00 GMT",2021-02-16 21:27:00+00:00
